In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
plot_params = dict(surface='white', hemi='both', verbose=0)

# Forward Model

In [2]:
info = get_info()
info['sfreq'] = 100
fwd = create_forward_model(info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.0s remaining:    1.8s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.0s remaining:    0.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


# Simulate

In [3]:
n_samples = 100
settings = dict(duration_of_trial=0.1, target_snr=(0.5, 10))

sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)
sim_dense = util.convert_simulation_temporal_to_single(sim_lstm)

sim_lstm_test = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=1000)
sim_dense_test = util.convert_simulation_temporal_to_single(sim_lstm_test)

Simulate Source


  0%|          | 0/100 [00:00<?, ?it/s]

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/100 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


...

Converting Source Data to mne.SourceEstimate object


  0%|          | 0/1000 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


  0%|          | 0/1000 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


# Keras-Tuner

In [11]:
import tensorflow as tf

a = np.random.rand(13, 14, 15)
b = 1+ np.random.rand(13, 14, 15)

a = tf.convert_to_tensor(a)
b = tf.convert_to_tensor(b)


from esinet.evaluate import tf_auc_metric
tf_auc_metric(a,b)

go next
[<tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor:

AttributeError: 'list' object has no attribute 'dtype'

In [105]:
import keras_tuner as kt
import tensorflow as tf
from esinet.net import build_nas_lstm
from esinet.evaluate import modified_auc_metric

epochs = 2

tuner = kt.BayesianOptimization(
    build_nas_lstm,
    objective='val_loss', #kt.Objective('auc_loss', direction="max"),
    max_trials=99999,
    executions_per_trial=3,
    directory='keras-tuner',
    project_name=f'bayes_auc'
    )    
# tuner = kt.RandomSearch(
#     build_nas_lstm,
#     objective='val_loss',
#     max_trials=5000, 
#     directory='keras-tuner',
#     executions_per_trial=3,
#     project_name=f'post-holiday_run',
#     )
callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=15, min_delta=0.0001)]
net = Net(fwd)
x_train, y_train = net.prep_data(sim_lstm)
x_val, y_val = net.prep_data(sim_lstm_test)
tuner.search(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val), 
    batch_size=8, callbacks=callbacks, shuffle=True)

best_model = tuner.get_best_models()[0]

INFO:tensorflow:Reloading Oracle from existing project keras-tuner\bayes_auc\oracle.json

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
lstm_layers       |2                 |?                 
dense_layers      |3                 |?                 
Momentum          |0.56528           |?                 
Nesterov          |0                 |?                 
learning_rate     |0.01              |?                 
Optimizer         |2                 |?                 

Epoch 1/2


OperatorNotAllowedInGraphError: in user code:

    C:\Users\lukas\virtualenvs\esienv\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    C:\Users\lukas\virtualenvs\esienv\lib\site-packages\sklearn\metrics\_ranking.py:83 auc  *
        check_consistent_length(x, y)
    C:\Users\lukas\virtualenvs\esienv\lib\site-packages\sklearn\utils\validation.py:317 check_consistent_length  *
        uniques = np.unique(lengths)
    <__array_function__ internals>:5 unique  **
        
    C:\Users\lukas\virtualenvs\esienv\lib\site-packages\numpy\lib\arraysetops.py:261 unique
        ret = _unique1d(ar, return_index, return_inverse, return_counts)
    C:\Users\lukas\virtualenvs\esienv\lib\site-packages\numpy\lib\arraysetops.py:322 _unique1d
        ar.sort()
    C:\Users\lukas\virtualenvs\esienv\lib\site-packages\tensorflow\python\framework\ops.py:900 __bool__
        self._disallow_bool_casting()
    C:\Users\lukas\virtual

# Visualize Keras Tuner results

In [ ]:
import os
import json
import pandas as pd

base_path = 'keras-tuner/post-holiday_run_bayes'
folders = os.listdir(base_path)
folders = [folder for folder in folders if folder.startswith('trial_')]
hyperparams = []
scores = []
for folder in folders:
    # Load trial data
    with open(base_path + '/' + folder + '/trial.json') as json_file:
        data = json.load(json_file)
    try:
        # extract score
        scores.append( data['metrics']['metrics']['val_loss']['observations'][0]['value'][0] )
        # extract hyperparameters
        hyperparams.append( data['hyperparameters']['values'] )
    except:
        pass

dfs = []
for i, (score, hyper) in enumerate(zip(scores, hyperparams)):
    df = pd.DataFrame.from_dict(hyper, orient='index').T
    df['score'] = score
    dfs.append(df)

df = pd.concat(dfs).sort_values('score')

# df.to_excel(r'C:\Users\lukas\Dokumente\projects\esinet\keras-tuner\weekend_run_bayes.xlsx')
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import spearmanr
import pprint; pp = pprint.PrettyPrinter(indent=4)
%matplotlib qt

x = df['dropout_dense_l-0'].values
y = df['score'].values

x = x[~pd.isnull(y)]
y = y[~pd.isnull(y)] 

y = y[~pd.isnull(x)]
x = x[~pd.isnull(x)]

step = 0.1
x = np.digitize(x,bins=np.arange(0, 1, step)) * step

plt.figure()
plt.scatter(x, y)
corr = spearmanr(x,y)[0]
plt.title(f'Corr: {corr}')
categorical_dict = dict()
if type(x) == np.ndarray and len(set(x)) < len(x)*0.8:
    categories = set(x)
    for cat in categories:
        sub_values = y[x==cat]
        median = np.median(sub_values)
        minimum_value = np.min(sub_values)
        perc = np.percentile(sub_values, 5)
        categorical_dict[cat] = f'Median: {median:.4f}, min: {minimum_value:.4f}, best 5%: {perc:.4f}'
else:
    categories = []
pp.pprint(categorical_dict)
